In [1]:
import carla
import time
import datetime
import random
import pandas as pd

# ✅ Connect to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(5.0)
world = client.get_world()

# ✅ Get map and spawn points
map = world.get_map()
spawn_points = map.get_spawn_points()
start_point = random.choice(spawn_points)

# ✅ Spawn the vehicle
vehicle_bp = world.get_blueprint_library().filter('*firetruck*')[0]
vehicle = world.try_spawn_actor(vehicle_bp, start_point)

if not vehicle:
    print("❌ Failed to spawn vehicle.")
    exit()

print(f"🚗 Vehicle spawned at {start_point.location}")

# ✅ Disable autopilot (We will manually apply noisy waypoints)
vehicle.set_autopilot(False)

# ✅ Attach Collision Sensor
collision_bp = world.get_blueprint_library().find('sensor.other.collision')
collision_sensor = world.spawn_actor(collision_bp, carla.Transform(), attach_to=vehicle)

# ✅ Set up Excel File for Results
excel_filename = "waypoint_noise_incidents.xlsx"

# ✅ Create a list to store results (faster than DataFrame updates)
data_records = []

# ✅ Waypoint Noise Model (More Intense Noise)
def noisy_waypoint(original_waypoint, noise_level=1.5):  # Noise in meters
    noisy_x = original_waypoint.transform.location.x + random.uniform(-noise_level, noise_level)
    noisy_y = original_waypoint.transform.location.y + random.uniform(-noise_level, noise_level)
    noisy_z = original_waypoint.transform.location.z  # Keeping altitude the same
    return carla.Transform(carla.Location(x=noisy_x, y=noisy_y, z=noisy_z), original_waypoint.transform.rotation)

# ✅ Track simulation time
start_time = time.time()
crash_count = 0  # Track number of crashes

# ✅ Collision detection function (Reset position instead of stopping)
def collision_callback(event):
    global crash_count
    print("💥 Collision detected!")

    crash_count += 1  # Increment crash count

    # Log collision event
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    data_records.append({
        "Timestamp": timestamp,
        "Waypoint_X": None,  # No waypoint update here
        "Waypoint_Y": None,
        "Waypoint_Z": None,
        "Noisy_Waypoint_X": None,
        "Noisy_Waypoint_Y": None,
        "Noisy_Waypoint_Z": None,
        "Collision": "Yes",
        "Collision_Object": str(event.other_actor)
    })

    # Reset vehicle to a new random spawn point
    reset_location = random.choice(spawn_points)
    vehicle.set_transform(reset_location)

    print("🔄 Vehicle reset after collision. Continuing experiment.")

# ✅ Cleanup function
def cleanup():
    print("🛑 Cleaning up...")

    # Convert collected data into a DataFrame and save to Excel
    df = pd.DataFrame(data_records)
    df.to_excel(excel_filename, index=False)
    print(f"✅ Incident data saved to {excel_filename}")
    
    print(f"📊 Total crashes recorded: {crash_count}")

    if collision_sensor:
        collision_sensor.destroy()
    if vehicle:
        vehicle.destroy()
    print("✅ Vehicle & sensors destroyed. Exiting program.")

# ✅ Start collision sensor
collision_sensor.listen(collision_callback)

# ✅ Move vehicle along waypoints with noise
print("🚦 Vehicle navigating with noisy waypoints...")

try:
    while True:
        current_location = vehicle.get_location()
        next_waypoint_options = map.get_waypoint(current_location).next(5.0)

        if not next_waypoint_options:
            print("⚠️ No further waypoints found! Resetting vehicle...")
            vehicle.set_transform(random.choice(spawn_points))
            continue
        
        next_waypoint = next_waypoint_options[0]  # Get next waypoint 5 meters ahead

        # Apply noise to waypoint
        noisy_transform = noisy_waypoint(next_waypoint, noise_level=2.0)  # 2m max deviation
        vehicle.set_transform(noisy_transform)  # Move vehicle to noisy waypoint

        # Log waypoint data
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        data_records.append({
            "Timestamp": timestamp,
            "Waypoint_X": next_waypoint.transform.location.x,
            "Waypoint_Y": next_waypoint.transform.location.y,
            "Waypoint_Z": next_waypoint.transform.location.z,
            "Noisy_Waypoint_X": noisy_transform.location.x,
            "Noisy_Waypoint_Y": noisy_transform.location.y,
            "Noisy_Waypoint_Z": noisy_transform.location.z,
            "Collision": "No",
            "Collision_Object": "None"
        })

        print(f"📍 Noisy Waypoint Applied: {noisy_transform.location}")

        time.sleep(0.5)  # Pause between movements

        # Stop after 1 minute
        if time.time() - start_time >= 60:
            print("⏳ Time limit reached (1 minute). Stopping simulation...")
            cleanup()
            break

except KeyboardInterrupt:
    print("\n🛑 Manual stop detected! Cleaning up before exiting...")
    cleanup()


🚗 Vehicle spawned at Location(x=85.982246, y=66.358490, z=0.600000)
🚦 Vehicle navigating with noisy waypoints...
📍 Noisy Waypoint Applied: Location(x=80.380157, y=64.680191, z=0.000000)
📍 Noisy Waypoint Applied: Location(x=75.164398, y=64.582939, z=0.000000)
📍 Noisy Waypoint Applied: Location(x=70.137314, y=67.691681, z=0.000000)
📍 Noisy Waypoint Applied: Location(x=65.939392, y=65.478226, z=0.000000)
📍 Noisy Waypoint Applied: Location(x=60.636887, y=66.564018, z=0.000000)
📍 Noisy Waypoint Applied: Location(x=57.242817, y=67.075600, z=0.000000)
📍 Noisy Waypoint Applied: Location(x=53.939697, y=67.834328, z=0.000000)
📍 Noisy Waypoint Applied: Location(x=50.645508, y=64.680412, z=0.000000)
📍 Noisy Waypoint Applied: Location(x=45.674507, y=60.550041, z=0.000000)
📍 Noisy Waypoint Applied: Location(x=43.592354, y=57.565220, z=0.000000)
📍 Noisy Waypoint Applied: Location(x=44.747883, y=52.261951, z=0.000000)
📍 Noisy Waypoint Applied: Location(x=44.389713, y=47.381306, z=0.000000)
📍 Noisy Way

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

# ✅ Load the Excel file
excel_filename = "waypoint_noise_incidents.xlsx"
df = pd.read_excel(excel_filename)

# ✅ Print basic statistics
total_waypoints = len(df[df["Collision"] == "No"])
total_crashes = len(df[df["Collision"] == "Yes"])

print(f"📊 Total waypoints visited: {total_waypoints}")
print(f"💥 Total crashes recorded: {total_crashes}")




📊 Total waypoints visited: 119
💥 Total crashes recorded: 16
